In [1]:
import sys, os
import pandas as pd
import numpy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
sys.argv[1] = 'dating.csv'
sys.argv[2] = 'dating-binned.csv'
# Read the data
df = pd.read_csv(sys.argv[1])

In [3]:
discrete_columns = ['gender', 'race', 'race_o', 'samerace', 'field', 'decision']
all_columns = df.columns.values.tolist()
continuous_valued_columns = [item for item in all_columns if item not in discrete_columns]
print all_columns
print continuous_valued_columns

['gender', 'age', 'age_o', 'race', 'race_o', 'samerace', 'importance_same_race', 'importance_same_religion', 'field', 'pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_important', 'sincere_important', 'intelligence_important', 'funny_important', 'ambition_important', 'shared_interests_important', 'attractive', 'sincere', 'intelligence', 'funny', 'ambition', 'attractive_partner', 'sincere_partner', 'intelligence_parter', 'funny_partner', 'ambition_partner', 'shared_interests_partner', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'interests_correlate', 'expected_happy_with_sd_people', 'like', 'decision']
['age', 'age_o', 'importance_same_race', 'importance_same_religion', 'pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious', 'pref_o_

In [4]:
(row, col) = df.shape
age_range = [18.0, 58.0]
pref_score = [0.0, 1.0]
score = [0.0, 10.0]
corr_range = [-1.00, 1.00]

bin_N = 5
#bin_value = [i for i in range(bin_N)]
bin_seg = [1.000 * i/bin_N for i in range(0, bin_N + 1)]
print bin_seg

age = ['age', 'age_o']
corr = ['interests_correlate']
preference_scores_of_participant = \
['attractive_important', 'sincere_important', 'intelligence_important',\
 'funny_important', 'ambition_important', 'shared_interests_important']

preference_scores_of_partner = \
['pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', \
 'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests']

continuous_valued_columns_bins = {}

# Segment the bins
for field in continuous_valued_columns:
    continuous_valued_columns_bins[field] = []
    if field in age:
        for i in range(0, bin_N + 1):
            continuous_valued_columns_bins[field].append(age_range[0] + bin_seg[i] * (age_range[1] - age_range[0]))
    elif field in corr:
        for i in range(0, bin_N + 1):
            continuous_valued_columns_bins[field].append(corr_range[0] + bin_seg[i] * (corr_range[1] - corr_range[0]))
    elif field in preference_scores_of_participant or field in preference_scores_of_partner:
        for i in range(0, bin_N + 1):
            continuous_valued_columns_bins[field].append(pref_score[0] + bin_seg[i] * (pref_score[1] - pref_score[0]))
    else:
        for i in range(0, bin_N + 1):
            continuous_valued_columns_bins[field].append(score[0] + bin_seg[i] * (score[1] - score[0]))

print continuous_valued_columns_bins
print len(continuous_valued_columns_bins)

[0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
{'art': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'age_o': [18.0, 26.0, 34.0, 42.0, 50.0, 58.0], 'intelligence': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'sincere': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'expected_happy_with_sd_people': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'attractive_important': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'sincere_important': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'shared_interests_important': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'sincere_partner': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'pref_o_ambitious': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'shared_interests_partner': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'funny': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'pref_o_intelligence': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'yoga': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'hiking': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'dining': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'tv': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0], 'pref_o_shared_interests': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'pref_o_funny': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'clubbing': [0.0

In [5]:
# Dictionary of the numbers ine ach bin
continuous_valued_columns_seg = {}
# Initalize the dict
for field in continuous_valued_columns:
    continuous_valued_columns_seg[field] = [0 for i in range(bin_N)]

for i in range(row):
    for field in continuous_valued_columns:
        # Find the bin
        for j in range(0, bin_N):
            # Corner Case
            if j == bin_N - 1:
                if float(continuous_valued_columns_bins[field][j]) <= float(df[field][i]):
                    df.loc[i,field] = int(j)
                    continuous_valued_columns_seg[field][j] += 1
                    break 
            else:
                if float(continuous_valued_columns_bins[field][j]) <= float(df[field][i]) < float(continuous_valued_columns_bins[field][j + 1]):
                    df.loc[i,field] = int(j)
                    continuous_valued_columns_seg[field][j] += 1
                    break

In [6]:
df = df.astype('int64')
df.to_csv(sys.argv[2], index=False)
print continuous_valued_columns_seg

{'art': [90, 624, 1262, 1863, 2905], 'age_o': [2990, 3495, 238, 16, 5], 'intelligence': [0, 127, 409, 2157, 4051], 'sincere': [0, 57, 228, 1322, 5137], 'expected_happy_with_sd_people': [107, 841, 2289, 2808, 699], 'attractive_important': [2990, 3103, 545, 87, 19], 'sincere_important': [3775, 2884, 85, 0, 0], 'shared_interests_important': [5745, 999, 0, 0, 0], 'sincere_partner': [37, 166, 818, 2646, 3077], 'pref_o_ambitious': [6258, 486, 0, 0, 0], 'shared_interests_partner': [251, 994, 2091, 2247, 1161], 'funny': [0, 9, 84, 1000, 5651], 'pref_o_intelligence': [2515, 4091, 138, 0, 0], 'yoga': [1332, 1806, 1237, 1355, 1014], 'hiking': [364, 1389, 1321, 1743, 1927], 'dining': [29, 68, 602, 1864, 4181], 'tv': [670, 1034, 1608, 1921, 1511], 'pref_o_shared_interests': [5671, 1073, 0, 0, 0], 'pref_o_funny': [3924, 2760, 60, 0, 0], 'clubbing': [651, 824, 1196, 2040, 2033], 'sports': [259, 907, 1148, 1626, 2804], 'ambition_partner': [34, 237, 1309, 2784, 2380], 'intelligence_parter': [14, 76, 65

In [7]:
keylist = continuous_valued_columns_seg.keys()
keylist.sort()

for field in keylist:
    print field, ':', continuous_valued_columns_seg[field]

age : [3032, 3491, 196, 20, 5]
age_o : [2990, 3495, 238, 16, 5]
ambition : [0, 225, 697, 1867, 3955]
ambition_important : [6246, 498, 0, 0, 0]
ambition_partner : [34, 237, 1309, 2784, 2380]
art : [90, 624, 1262, 1863, 2905]
attractive : [0, 131, 726, 3260, 2627]
attractive_important : [2990, 3103, 545, 87, 19]
attractive_partner : [87, 519, 1661, 2743, 1734]
clubbing : [651, 824, 1196, 2040, 2033]
concerts : [55, 516, 1199, 2177, 2797]
dining : [29, 68, 602, 1864, 4181]
exercise : [223, 895, 1266, 1959, 2401]
expected_happy_with_sd_people : [107, 841, 2289, 2808, 699]
funny : [0, 9, 84, 1000, 5651]
funny_important : [3934, 2750, 60, 0, 0]
funny_partner : [100, 413, 1471, 2723, 2037]
gaming : [1573, 1877, 1453, 1292, 549]
hiking : [364, 1389, 1321, 1743, 1927]
importance_same_race : [2201, 1629, 897, 886, 1131]
importance_same_religion : [2507, 1475, 977, 848, 937]
intelligence : [0, 127, 409, 2157, 4051]
intelligence_important : [2504, 4105, 135, 0, 0]
intelligence_parter : [14, 76, 65